In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from dateutil.parser import parse
#Download chromedriver and write its path to chromedriver variable
chromedriver = "/home/nebojsa/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
def contest_page(driver):
    
    y = driver.find_elements_by_tag_name('p')

    fields_of_interest = ['Category:', 'Eligibility:', 'Start Date:',
                          'Sweepstakes Prizes:', 'Additional Comments:']

    next_item = None
    resp = {}
    for item in y:
        if item.text in fields_of_interest:
            next_item = item.text.lower().split(':')[0]
        elif next_item is not None:
            if next_item == 'category':
                resp[next_item] = item.text.split(', ')
            else:   
                resp[next_item] = item.text
            next_item = None


    return resp

In [3]:
def get_sweepstakes():

    driver = webdriver.Chrome(chromedriver)
    driver.get('https://www.thebalance.com/new-sweepstakes-and-contests-896980')

    hits = driver.find_elements_by_tag_name('li')

    data = []

    for idx, row in enumerate(hits):
        s = row.find_elements_by_tag_name('strong')
        if len(s) == 0:
            continue
        else:
            driver_temp = webdriver.Chrome(chromedriver)
            html = row.get_attribute("outerHTML")
            link = html.split('</strong>')[0].split('''<a href="''')[1].split('''"''')[0]
            driver_temp.get(link)
            resp = contest_page(driver_temp)
            driver_temp.quit()
            entry_freq = html.split('</strong>')[2].split('<br>')[0]
            end_date = html.split('</strong>')[3].split('<br>')[0]
            content = html.split('</strong>')[3].split('<br>')[1].split('</li>')[0]
            resp.update({'title': s[0].text,
                         'entry_freq': entry_freq,
                         'end_date': end_date,
                         'content': s[0].text})
            data.append(resp)

    driver.quit()
    return data

In [ ]:
data = get_sweepstakes()
df = pd.DataFrame(data)
file = '/home/nebojsa/Documents/sweepstakes.csv'
df.to_csv(file)
#df = pd.read_csv('/home/nebojsa/Documents/sweepstakes.csv')